#### Setting environments

In [1]:
import torch, os, gc, itertools
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [2]:
# Set options
embed_ver = ["clstm", "esm2", "bert", "t5"]

# model options
layer_num = 3
unit_decrease = 2

data_path = "../data/"
model_path = f"../model/emb_gen-ensem/"
result_path = f"../result/prd-emb_gen-ensem/"
os.makedirs(result_path, exist_ok=True)

batch_size = 256
col_str = ['file_id', 'organism', 'locus_tag', 'ess']

In [3]:
# Set data list for test dataset
ts_data = {
    "data1": ["C018"],  # "Escherichia coli K-12 BW25113"
    "data2": ["C016"],  # "Escherichia coli K-12 MG1655"
    "data3": ["O046"],  # "synthetic bacterium JCVI-Syn3A"
    "data4": ["C048"],  # Bacteroides thetaiotaomicron VPI-5482
    "data5": ["C050"]  # Salmonella enterica subsp. enterica serovar Typhimurium str. 14028S
}

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# Define function to record perfomance result
def record_perform(comb_ver, file_id, organ, y_real, y_conf, y_prd):
    y_real = y_real.cpu().numpy()
    y_conf = y_conf.cpu().numpy()
    y_prd = y_prd.cpu().numpy()
    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "comb": [comb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result


In [6]:
# Set model architecture
class Classifier(nn.Module):
    def __init__(self, input_size, num_layers, unit_decrease):
        super(Classifier, self).__init__()
        layers = [nn.BatchNorm1d(input_size), nn.Dropout(0.5)]
        in_dim = input_size
        out_dim = 1024
        for i in range(num_layers):            
            out_dim = max(2, out_dim // unit_decrease)
            layers.append(nn.Linear(in_dim, out_dim))
            self.initialize_weights(layers[-1])
            layers.append(nn.GELU())
            in_dim = out_dim
        layers.append(nn.Linear(out_dim, 1))
        self.cls_block = nn.Sequential(*layers)
        
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        return self.cls_block(x)

#### Evaluation

In [7]:
dfs = [(e_ver, pd.read_csv(data_path + f"data-emb_gen-{e_ver}-ts.csv")) for e_ver in embed_ver]

In [8]:
df_eval = pd.DataFrame()

for r in range(2, len(dfs) + 1):
    combs = list(itertools.combinations(dfs, r))
    
    for comb in combs:
        comb_ver = "_".join([df[0] for df in comb])
        print(f"\n>>>> {comb_ver} <<<<")
        
        # merge dataset
        data = comb[0][1]
        for df in comb[1:]:
            data = pd.merge(data, df[1], on=col_str, suffixes=("", f"_{df[0]}"))
        
        display("Raw data:", data)
        
        # calculate mean of confidences
        col_num = [col for col in data.columns if col not in col_str]
    
        # get test datasets
        loc_ts = {}
        data_ts = {}
        org_ts = {}
        for ts_ver, ids in ts_data.items():
            # get test sample locations
            loc_ts[ts_ver] = data['file_id'].isin(ids)
            # get test samples
            data_ts[ts_ver] = data[loc_ts[ts_ver]]
            org = []
            # get test organism list
            for i in ids:
                organ = data_ts[ts_ver]['organism'][data_ts[ts_ver]['file_id'] == i].to_list()
                if len(organ) > 0:
                    org.append(organ[0])
            org_ts[ts_ver] = org    
            print("Test dataset(" + ts_ver + "):", data_ts[ts_ver].shape)
        print("Test organism:", org_ts, len(org_ts))
        
        # split info.& inputs & labels of the test datasets
        info_ts = {}
        y_ts = {}
        test_loader = {}
        for ts_ver, df in data_ts.items():
            info_ts[ts_ver] = df[col_str]
            X_ts = torch.tensor(df.iloc[:, len(col_str):].astype('float32').values)
            y_ts[ts_ver] = torch.tensor(df['ess'].astype('float32').values)
            print("Splited test dataset(" + ts_ver + "):", X_ts.shape, y_ts[ts_ver].shape)                    
            # generate dataloader by the test datasets
            dataset_ts = TensorDataset(X_ts, y_ts[ts_ver])
            test_loader[ts_ver] = DataLoader(dataset_ts, batch_size=batch_size, shuffle=False)
        
        # get the total test dataset info.
        loc_ts_all = [sum(loc) >= 1 for loc in zip(*loc_ts.values())]
        info_ts_all = data.loc[loc_ts_all, col_str]
        X_ts_all = data.iloc[:, len(col_str):]
        X_ts_all = torch.tensor(X_ts_all.loc[loc_ts_all, :].astype('float32').values)
        y_ts_all = torch.tensor(data.loc[loc_ts_all, 'ess'].astype('float32').values)        
        print("Splited test dataset(all):", X_ts_all.shape, y_ts_all.shape)
        
        # generate dataloader of total test dataset
        test_all_dataset = TensorDataset(X_ts_all, y_ts_all)
        test_all_loader = DataLoader(test_all_dataset, batch_size=256, shuffle=False)
        
        ## Test model ##
        # set model name
        model_name = f"emb_gen-ensem-{comb_ver}"
        print(f"\n===== Test model: {model_name} ====")
        # generate model instance
        model = Classifier(
            input_size=X_ts_all.shape[-1],
            num_layers=layer_num,
            unit_decrease=unit_decrease
        ).to(device)

        # load model weight
        model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))
        model.eval()
        
        # evaluations by test dataset
        for ts_ver, ids in ts_data.items():
            all_preds = []
            all_labels = []
            with torch.no_grad():
                for X_batch, y_batch in test_loader[ts_ver]:
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)
                    preds = model(X_batch).squeeze()
                    all_preds.append(preds.cpu())
                    all_labels.append(y_batch.cpu())
            
            # concatenate results to one tensor
            all_preds = torch.cat(all_preds)
            all_labels = torch.cat(all_labels)

            # convert logits to confidences & classes
            prd_conf = torch.sigmoid(all_preds)
            prd_cls = (prd_conf >= 0.5).int()
            # performances by testset
            perform = record_perform(
                comb_ver=f"{comb_ver}",
                test_ver=ts_ver,
                file_id="+".join(ids),
                organ="+".join(org_ts[ts_ver]),
                y_real=y_ts[ts_ver],
                y_conf=prd_conf,
                y_prd=prd_cls,
            )
            display(perform)
            df_eval = pd.concat([df_eval, perform], ignore_index=True)
        
        
        # model evaluation on the total test dataset
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for X_batch, y_batch in test_all_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                preds = model(X_batch).squeeze()
                all_preds.append(preds.cpu())
                all_labels.append(y_batch.cpu())
        
        # concatenate results to one tensor
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        # convert logits to confidences & classes
        prd_conf = torch.sigmoid(all_preds)
        prd_cls = (prd_conf >= 0.5).int()

        # performances on total testset
        perform = record_perform(
            comb_ver=f"{comb_ver}",
            test_ver="test_all",
            file_id="+".join([i for ids in ts_data.values() for i in ids]),
            organ="+".join([org for orgs in org_ts.values() for org in orgs]),
            y_real=y_ts_all,
            y_conf=prd_conf,
            y_prd=prd_cls
        )
        display(perform)
        df_eval = pd.concat([df_eval, perform], ignore_index=True)

        # concatenate the protein info. & predicted confidences
        df_prd = pd.DataFrame(prd_conf, columns=["conf"], index=info_ts_all.index)
        df_prd = pd.concat([info_ts_all, df_prd], axis=1)

        # save the model prediction result
        df_prd.to_csv(result_path + f"prd-{model_name}.csv", index=False)
    
        gc.collect()

# save the model perfomance result
display("Model performance:", df_eval)
df_eval.to_csv(f"../result/eval-emb_gen-ensem.csv", index=False)


>>>> clstm_esm2 <<<<


'Raw data:'

,file_id,organism,locus_tag,ess,0,1,2,3,4,5,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0001,0,-1.271414,0.000099,0.076290,-0.000540,-0.002149,-0.002770,...,-0.155313,1.499200,-1.622623,1.140054,-0.579188,-0.201200,-0.181841,2.446096,1.391733,-0.146903
1,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0002,0,0.080849,0.000191,0.066948,-0.001360,-0.001139,-0.002485,...,-0.441345,-0.216852,-0.173726,0.456722,-0.253636,0.488008,-0.237100,0.032918,-0.110359,0.293158
2,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0003,0,0.049742,0.000037,0.063013,-0.003911,-0.002671,-0.000340,...,-0.291001,-0.496596,-0.240145,0.931340,0.250903,0.500919,-0.206658,0.055057,0.349885,0.051546
3,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0004,0,0.438240,0.000130,0.061992,-0.002766,-0.001883,-0.001800,...,0.079341,-0.345971,-0.169741,0.699338,-0.327057,0.669865,-0.155763,0.371676,-0.280873,-0.052240
4,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0005,0,0.647014,0.000091,0.058150,-0.003637,-0.002623,-0.002049,...,0.157999,-0.030111,0.223549,1.058194,-0.627709,-0.035077,0.126186,0.007433,-0.104928,-0.069931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19398,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0913,1,0.244533,0.000047,0.060919,-0.003986,-0.002723,-0.000765,...,-0.052086,-0.375514,-0.349227,1.061728,-0.046756,0.376358,0.299031,0.954350,1.110562,0.166870
19399,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0918,1,0.446268,0.000114,0.060191,-0.002962,-0.002494,-0.002787,...,-0.216774,-0.193204,-0.085630,0.603998,0.047384,0.273780,0.067045,-0.170716,0.290532,0.482953
19400,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0930,1,-1.313249,0.000093,0.077817,-0.000610,-0.002145,-0.002522,...,-0.729781,-0.557699,-0.633800,0.923602,-1.082554,0.885507,-1.106922,-1.015008,-0.201546,-0.682305
19401,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0931,1,0.339057,0.000057,0.060354,-0.003875,-0.002801,-0.001344,...,0.835866,-0.616349,-0.738524,-0.103508,0.191622,0.856668,-0.864706,0.502746,0.409977,0.147629


Test dataset(data1): (4313, 2308)
Test dataset(data2): (4333, 2308)
Test dataset(data3): (458, 2308)
Test dataset(data4): (4825, 2308)
Test dataset(data5): (5474, 2308)
Test organism: {'data1': ['Escherichia coli K-12 BW25113'], 'data2': ['Escherichia coli K-12 MG1655'], 'data3': ['synthetic bacterium JCVI-Syn3A'], 'data4': ['Bacteroides thetaiotaomicron VPI-5482'], 'data5': ['Salmonella enterica subsp. enterica serovar Typhimurium str. 14028S']} 5
Splited test dataset(data1): torch.Size([4313, 2304]) torch.Size([4313])
Splited test dataset(data2): torch.Size([4333, 2304]) torch.Size([4333])
Splited test dataset(data3): torch.Size([458, 2304]) torch.Size([458])
Splited test dataset(data4): torch.Size([4825, 2304]) torch.Size([4825])
Splited test dataset(data5): torch.Size([5474, 2304]) torch.Size([5474])
Splited test dataset(all): torch.Size([19403, 2304]) torch.Size([19403])

===== Test model: emb_gen-ensem-clstm_esm2 ====


/tmp/ipykernel_4901/3219965690.py:76: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))


...

,comb,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm_esm2_bert_t5,data4,C048,Bacteroides thetaiotaomicron VPI-5482,309,210,4290,16,0.731383,0.953161,0.732227,0.595376,0.950769,0.996284,0.953333,0.979275,0.838159


,comb,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm_esm2_bert_t5,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,95,548,4813,18,0.326123,0.896602,0.251323,0.147745,0.840708,0.996274,0.89778,0.883667,0.104937


,comb,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm_esm2_bert_t5,test_all,C018+C016+O046+C048+C050,Escherichia coli K-12 BW25113+Escherichia coli...,1192,1487,16496,384,0.533304,0.904341,0.560282,0.444942,0.756345,0.977251,0.917311,0.915194,0.552855


'Model performance:'

,comb,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm_esm2,data1,C018,Escherichia coli K-12 BW25113,249,371,3644,49,0.537220,0.902620,0.542484,0.401613,0.835570,0.986732,0.907597,0.920239,0.645256
1,clstm_esm2,data2,C016,Escherichia coli K-12 MG1655,269,358,3677,29,0.585527,0.910685,0.581622,0.429027,0.902685,0.992175,0.911276,0.961634,0.741339
2,clstm_esm2,data3,O046,synthetic bacterium JCVI-Syn3A,154,0,0,304,0.000000,0.336245,0.503268,1.000000,0.336245,0.000000,0.000000,NaN,NaN
3,clstm_esm2,data4,C048,Bacteroides thetaiotaomicron VPI-5482,309,182,4318,16,0.754674,0.958964,0.757353,0.629328,0.950769,0.996308,0.959556,0.982608,0.877225
4,clstm_esm2,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,96,543,4818,17,0.331339,0.897698,0.255319,0.150235,0.849558,0.996484,0.898713,0.908792,0.110601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,clstm_esm2_bert_t5,data2,C016,Escherichia coli K-12 MG1655,355,357,3750,27,0.643422,0.914458,0.648995,0.498596,0.929319,0.992851,0.913075,0.964272,0.773961
62,clstm_esm2_bert_t5,data3,O046,synthetic bacterium JCVI-Syn3A,184,0,0,274,0.000000,0.401747,0.573209,1.000000,0.401747,0.000000,0.000000,NaN,NaN
63,clstm_esm2_bert_t5,data4,C048,Bacteroides thetaiotaomicron VPI-5482,309,210,4290,16,0.731383,0.953161,0.732227,0.595376,0.950769,0.996284,0.953333,0.979275,0.838159
64,clstm_esm2_bert_t5,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,95,548,4813,18,0.326123,0.896602,0.251323,0.147745,0.840708,0.996274,0.897780,0.883667,0.104937
